In [150]:
import jieba
import numpy as np
#打開詞典文件，返回列表
def open_dict(Dict = 'haha', path=r'C:/Users/Java/python無聊爬蟲/語意處理/情緒分析/Textming'):
    path = path + '%s.txt' % Dict
    dictionary = open(path, 'r', encoding='utf-8')
    dict = []
    for word in dictionary:
        word = word.strip('\n').replace("\ufeff","")
        dict.append(word)
    return dict



def judgeodd(num):
    if (num % 2) == 0:
        return 'even'
    else:
        return 'odd'


#注意，這裡你要修改path路径。
deny_word = open_dict(Dict = '否定詞', path= r'C:/<file path>/')
posdict = open_dict(Dict = 'positive', path= r'C:/<file path>/')
negdict = open_dict(Dict = 'negative', path= r'C:/<file path>/')

degree_word = open_dict(Dict = '程度級別詞語', path= r'C:/Users/Java/python無聊爬蟲/語意處理/情緒分析/Textming/')
mostdict = degree_word[degree_word.index('extreme')+1 : degree_word.index('very')]#權重4，即在情感詞前乘以4
verydict = degree_word[degree_word.index('very')+1 : degree_word.index('more')]#權重3
moredict = degree_word[degree_word.index('more')+1 : degree_word.index('ish')]#權重2
ishdict = degree_word[degree_word.index('ish')+1 : degree_word.index('last')]#權重0.5



def sentiment_score_list(dataset):
    seg_sentence = dataset.split('。')

    count1 = []
    count2 = []
    for sen in seg_sentence: #循環遍歷每一個評論
        segtmp = jieba.lcut(sen, cut_all=False)  #把句子進行分詞，以列表的形式返回
        i = 0 #記錄掃描到的詞的位置
        a = 0 #紀錄情感詞的位置
        poscount = 0 #積極詞的第一次分值
        poscount2 = 0 #積極詞反轉後的分值
        poscount3 = 0 #積極詞的最後分值（包括驚嘆號的分值）
        negcount = 0
        negcount2 = 0
        negcount3 = 0
        for word in segtmp:
            if word in posdict:  # 判斷詞語是否是情感词
                poscount += 1
                c = 0
                for w in segtmp[a:i]:  # 掃描情感詞前的程度词
                    if w in mostdict:
                        poscount *= 4.0
                    elif w in verydict:
                        poscount *= 3.0
                    elif w in moredict:
                        poscount *= 2.0
                    elif w in ishdict:
                        poscount *= 0.5
                    elif w in deny_word:
                        c += 1
                if judgeodd(c) == 'odd':  # 掃描情感詞前的否定詞數
                    poscount *= -1.0
                    poscount2 += poscount
                    poscount = 0
                    poscount3 = poscount + poscount2 + poscount3
                    poscount2 = 0
                else:
                    poscount3 = poscount + poscount2 + poscount3
                    poscount = 0
                a = i + 1   # 情感詞的位置變化

            elif word in negdict:  # 消極情感的分析，與上面一致
                negcount += 1
                d = 0
                for w in segtmp[a:i]:
                    if w in mostdict:
                        negcount *= 4.0
                    elif w in verydict:
                        negcount *= 3.0
                    elif w in moredict:
                        negcount *= 2.0
                    elif w in ishdict:
                        negcount *= 0.5
                    elif w in degree_word:
                        d += 1
                if judgeodd(d) == 'odd':
                    negcount *= -1.0
                    negcount2 += negcount
                    negcount = 0
                    negcount3 = negcount + negcount2 + negcount3
                    negcount2 = 0
                else:
                    negcount3 = negcount + negcount2 + negcount3
                    negcount = 0
                a = i + 1
            elif word == '！' or word == '!':  ##判斷句子是否有感嘆號
                for w2 in segtmp[::-1]:  # 掃描感嘆號前的情感詞，發現后權值+2，然後退出循環
                    if w2 in posdict or negdict:
                        poscount3 += 2
                        negcount3 += 2
                        break
            i += 1 # 掃描詞位置前移


            # 以下是防止出現負數的情况
            pos_count = 0
            neg_count = 0
            if poscount3 < 0 and negcount3 > 0:
                neg_count += negcount3 - poscount3
                pos_count = 0
            elif negcount3 < 0 and poscount3 > 0:
                pos_count = poscount3 - negcount3
                neg_count = 0
            elif poscount3 < 0 and negcount3 < 0:
                neg_count = -poscount3
                pos_count = -negcount3
            else:
                pos_count = poscount3
                neg_count = negcount3

            count1.append([pos_count, neg_count])
        count2.append(count1)
        count1 = []

    return count2

def sentiment_score(senti_score_list):
    score = []
    for review in senti_score_list:
        score_array = np.array(review)
        Pos = np.sum(score_array[:, 0])
        Neg = np.sum(score_array[:, 1])
        AvgPos = np.mean(score_array[:, 0])
        AvgPos = float('%.1f'%AvgPos)
        AvgNeg = np.mean(score_array[:, 1])
        AvgNeg = float('%.1f'%AvgNeg)
        StdPos = np.std(score_array[:, 0])
        StdPos = float('%.1f'%StdPos)
        StdNeg = np.std(score_array[:, 1])
        StdNeg = float('%.1f'%StdNeg)
        score.append([Pos, Neg, AvgPos, AvgNeg, StdPos, StdNeg])
    return score


In [151]:
data = """院長加油，轉型正義，長期的司法体係，要改派院長不是人當的，長期低薪問题早就存在，人民要有共識，內憂外患，可悲阿，台湾人麥哥乱Y拉赖神加油拉！讃"""
data2= '我好開心啊，非常非常非常高興！今天我得了一百分，我很興奮開心，愉快，開心'
print(sentiment_score(sentiment_score_list(data)))

[[95, 46, 2.1, 1.0, 0.8, 1.0]]


# fb Comments test

In [ ]:
from pymongo import MongoClient
import re
client = MongoClient("mongodb://10.120.37.108:27017")
db = client["fb"]
collection_posts    = db["iw_posts"]
collection_comments = db["iw_comments"]

post    = collection_posts.find({"id":"46251501064_10154749205936065"},{"_id":0})
message = collection_comments.find({"post_id":"46251501064_10154749205936065"},{"message":1,"_id":0})
message_list = [i["message"] for i in message]

In [195]:
for i in post:
    print(i)

{'created_time': datetime.datetime(2017, 11, 22, 11, 30), 'message': '蔡英文總統針對獵雷艦案談話：\n\n親愛的國人同胞，大家好。針對臺灣社會非常關切的海軍獵雷艦案，今天下午，國防部已經提出相關的懲處名單。 \n\n做為三軍統帥，我對國軍的要求，就是勇敢面對，有錯就改，這才是國防改革的意義，這才是中華民國的國軍。 \n\n在這裡，我也要向國人說明針對本案，政府的三點立場。 \n\n第一、我們會釐清真相，嚴格究辦。 \n\n這項由前政府啟動的計畫，有明顯的疏失，無論是決策、招標、聯貸，以及履約的管理的過程，都應該被嚴格的檢視。 \n\n我要再次要求，本案所有的相關單位，務必坦蕩配合司法調查，在勿枉勿縱的原則下，如果有任何人涉弊，不論層級、不論是誰，絕不寬貸。 \n\n第二、務必要糾正錯誤，解決問題。 \n\n解決問題，是我們一貫的施政方針。如果過去做錯了，不論是何時發生的錯誤，我們就要確實檢討，積極解決、彌補錯誤。行政院已經完成本案的檢討報告，今天國防部也已經提出了懲處名單。 \n\n接下來，我要求國防部，務必全面檢討整個造艦案招標的過程，也必須在最短的時間內，對後續合約的事宜，做出專業的評估以及明快的決定，以確保國家利益。 \n\n更重要的是，我們一定要建立更完善的機制，把不適格的廠商排除在國防自主的計畫之外，杜絕錯誤再犯。危機就是轉機，我希望經過這一次的錯誤及失敗，國軍記取教訓，找出正確的方向與道路。 \n\n第三、 政府推動國艦國造的決心，不會改變。 \n\n走向國防自主的這條路，從來不是順風順水。但是，我們不會因為個案的挫折，影響國艦國造的決心。 \n\n我是三軍統帥，國軍有困難的時候，我一定會一起承擔。而國艦國造的工作，我們會更堅持到底。接下來，政府、國軍及產業，都要共同的投入，讓這項關係到國家安全的重大政策，克服萬難，繼續向前。 \n\n過去，國軍也有沱江艦及磐石艦成功的國艦國造經驗，我相信，只要方向正確、決心堅定，國艦國造的目標就會開花結果。 \n\n國防自主一定會成功，也一定要成功！謝謝大家，國軍加油。', 'id': '46251501064_10154749205936065', 'likes': 9851, 'shares': 810, 'link': 'https://www

In [196]:
s=[]
for i in message_list:
    sentence = re.findall('[!|！|\u4e00-\u9fa5]+', i)
    s.append(sentence)

In [198]:
positive_total_score = 0
negative_total_score = 0
count=0
for df in s:
    if df:
        cleaned_sentence = str(df).replace("[","").replace("]","").replace("'","").replace(",","").replace(" ","")
        positive_avg_score = sentiment_score(sentiment_score_list(cleaned_sentence))[0][2]
        negative_avg_score = sentiment_score(sentiment_score_list(cleaned_sentence))[0][3]
        
        positive_total_score += positive_avg_score 
        negative_total_score += negative_avg_score
        count+=1
print("總回文數=%s"%count)
print("正面分數=%s"%positive_total_score,"負面分數=%s"%negative_total_score)
print("正面平均分數=%s"%(positive_total_score/count),"負面平均分數=%s"%(negative_total_score/count))

總回文數=2193
正面分數=1900.1999999999978 負面分數=2215.599999999997
正面平均分數=0.866484268125854 負面平均分數=1.0103055175558582
